<a href="https://colab.research.google.com/github/Aiadevop/training_model_wine_quality/blob/main/DS_NL_Clasificaci%C3%B3n_MODELO_RANDOM_FOREST_Predicci%C3%B3n_calidad_del_vino.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import kagglehub
import matplotlib.pyplot as plt
import seaborn as sns
url="https://raw.githubusercontent.com/Aiadevop/training_model_wine_quality/refs/heads/main/data/df_cl_final.csv"
df_cl = pd.read_csv(url)
# Divido los datos para test y entrenamiento.
from sklearn.model_selection import train_test_split

# Dividir el DataFrame completo. Test_size 20,80 y random_state es la semilla
df_train, df_test = train_test_split(
    df_cl, test_size=0.2, random_state=42, stratify=df_cl['quality']
)

# Luego extraer X e y de cada DataFrame
X_train = df_train.drop('quality', axis=1)
y_train = df_train['quality']

X_test = df_test.drop('quality', axis=1)
y_test = df_test['quality']

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score

modelos = {
    "Random Forest": RandomForestClassifier(random_state=42)
}

resultados = []

for nombre, modelo in modelos.items():
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_test)
    y_proba = modelo.predict_proba(X_test)[:,1]

    acc = accuracy_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_proba)

    resultados.append({
        "Modelo": nombre,
        "Accuracy": acc,
        "ROC-AUC": roc_auc
    })

# Ordenar resultados por ROC-AUC descendente
resultados = sorted(resultados, key=lambda x: x["ROC-AUC"], reverse=True)

print(f"{'Modelo':<20} {'Accuracy':<10} {'ROC-AUC':<10}")
print("-"*45)
for r in resultados:
    print(f"{r['Modelo']:<20} {r['Accuracy']:<10.3f} {r['ROC-AUC']:<10.3f}")


Modelo               Accuracy   ROC-AUC   
---------------------------------------------
Random Forest        0.806      0.903     


<!-- No vale la pena optimizar parámetros ya está optimizado -->

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score
import numpy as np

# Definir el modelo base
rf = RandomForestClassifier(random_state=42)

# Espacio de búsqueda
param_dist = {
    'n_estimators': [100, 200, 300, 500],
    'max_depth': [None, 5, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', None]
}

# RandomizedSearchCV
random_search = RandomizedSearchCV(
    rf,
    param_distributions=param_dist,
    n_iter=30,
    scoring='roc_auc',
    cv=5,
    verbose=1,
    n_jobs=-1,
    random_state=42
)

# Entrenar
random_search.fit(X_train, y_train)

# Mejor modelo y predicción
best_rf = random_search.best_estimator_
y_pred = best_rf.predict(X_test)
y_proba = best_rf.predict_proba(X_test)[:, 1]

# Evaluar
acc = accuracy_score(y_test, y_pred)
roc = roc_auc_score(y_test, y_proba)

print("\n🔧 Mejor configuración encontrada:")
print(random_search.best_params_)

print(f"\n✅ Resultados del Random Forest optimizado:")
print(f"Accuracy: {acc:.3f}")
print(f"ROC-AUC:  {roc:.3f}")


Fitting 5 folds for each of 30 candidates, totalling 150 fits

🔧 Mejor configuración encontrada:
{'n_estimators': 300, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': None}

✅ Resultados del Random Forest optimizado:
Accuracy: 0.806
ROC-AUC:  0.905
